## **Описание задачи и датасета**

Датасет Titanic описывает пассажиров парохода «Титаник», затонувшего в 1912 году. Необходимо решить задачу бинарной классификации: по данным о пассажире нужно предсказать, выжил он или нет.

Переменные датасета titanic:

PassengerId — уникальный идентификатор пассажира (служебный, не признак реального мира).​

Survived — целевая переменная: 1 — пассажир выжил, 0 — погиб.

Pclass — класс билета / социально-экономический статус (1 — высший, 2 — средний, 3 — низший).​

Name — полное имя пассажира.

Sex — пол пассажира (male/female).

Age — возраст пассажира в годах, часть значений отсутствует.

SibSp — число братьев/сестер и супругов, находившихся на борту вместе с пассажиром.

Parch — число родителей и детей (включая приёмных), находившихся на борту вместе с пассажиром.​

Ticket — номер билета (строковый категориальный признак).​

Fare — стоимость билета (плата за проезд).

Cabin — номер каюты, очень много пропущенных значений.

Embarked — порт посадки: S — Саутгемптон, C — Шербур, Q — Куинстаун.

В датасете titanic информация о 891 пассажире титаника.

In [ ]:
# импорт необходимых библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# Загрузка данных
data = pd.read_csv('titanic.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# Разделение на признаки и целевую переменную
X = data.drop(columns=['Survived', 'PassengerId', 'Ticket', 'Name'])
y = data['Survived']

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     204 non-null    object 
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


In [21]:
# Заполнение пропусков - числовые признаки
for col in ['Age', 'Fare', 'SibSp', 'Parch', 'Pclass']:
    if X[col].isna().any():
        X[col] = X[col].fillna(X[col].median())

# Заполнение пропусков - категориальные признаки
for col in ['Embarked', 'Sex', 'Cabin']:
    if X[col].isna().any():
        X[col] = X[col].fillna(X[col].mode()[0])

In [ ]:
# бинарное кодирование для Sex
X['Sex'] = (X['Sex'] == 'female').astype(int)

In [ ]:
# One-hot кодирование для категориальных
X = pd.get_dummies(X, columns=['Embarked', 'Cabin'])

In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Обучение модели и оценка качества
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print("test accuracy:", model.score(X_val, y_val))

test accuracy: 0.8156424581005587
